<a href="https://colab.research.google.com/github/egmoturi/Data-pipelines-with-python-and-postgreSQL-Project/blob/main/Data_pipelines_with_python_and_postgreSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import psycopg2

Data extractrion

In [2]:
equipment_data = pd.read_csv('/content/equipment_sensor.csv')
network_data = pd.read_csv('/content/maintenance_records.csv')
maintenance_data = pd.read_csv('/content/network_sensor.csv')

Data transformation 

In [3]:
# Remove duplicates from data
equipment_data=pd.read_csv("/content/equipment_sensor.csv")
maintainance_data=pd.read_csv("/content/maintenance_records.csv")
network_sensor=pd.read_csv("/content/network_sensor.csv")

# Fix missing data
equipment_data = equipment_data.fillna(method='ffill')
maintainance_data=maintainance_data.fillna(method='ffill')
network_sensor=network_sensor.fillna(method='ffill')

# Normalize the sensor reading in equipment data
equipment_data['sensor_reading'] = (equipment_data['sensor_reading'] - equipment_data['sensor_reading'].mean()) / equipment_data['sensor_reading'].std()

In [4]:
# Aggregate the equipment data to get the total number of equipment failures
equipment_failures = equipment_data.groupby(['ID']).size().reset_index(name='total_failures')

# Aggregate the equipment data to get the average time between failures
equipment_data['datetime'] = pd.to_datetime(equipment_data['date'] + ' ' + equipment_data['time'])
equipment_data['time_between_failures'] = equipment_data.groupby(['ID'])['datetime'].diff()
equipment_data['time_between_failures'] = equipment_data['time_between_failures'].dt.total_seconds() / 3600
equipment_avg_time_between_failures = equipment_data.groupby(['ID'])['time_between_failures'].mean().reset_index(name='avg_time_between_failures')

In [5]:
# Join the datasets
df_joined = pd.concat([equipment_data, network_data, maintenance_data], axis=0)


In [6]:
#Data loading to postgreSQL database 
!curl ipecho.net/plain

35.188.229.105

In [7]:
pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import psycopg2

In [9]:
conn = psycopg2.connect(
    host="34.122.11.163",
    database="eric123",
    user="kamulu2",
    password="test"
)

In [10]:
# create a cursor object
#cur = conn.cursor()
#df_joined.to_sql(name="equipment_sensor,maintenance_records,network_sensor", con=conn, if_exists="append", index=False)
#cur.close()
#conn.close()



In [11]:
from sqlalchemy import create_engine
# Defining the database engine to be used to insert data
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username="kamulu2",
                        password="test",
                        ipaddress="34.122.11.163",
                        port="5432",
                        dbname="eric123"))
engine = create_engine(postgres_str)

In [12]:
df_joined.to_sql("equipment_sensor,maintenance_records,network_sensor",engine, if_exists="replace", index=False)

15